# GPT-Powered Chatbots: Enriching Conversations with Vector Database

### Connect to OpenAI API

In [81]:
import openai
openai.api_key = ""

prompt = "Who is the president of USA"

response = openai.Completion.create(model="text-davinci-003", prompt=prompt, temperature=0, max_tokens=1000)
response['choices'][0]['text']

'\n\nThe current President of the United States is Joe Biden.'

### What happens if we ask it something that it does not know, or it think it knows?

In [ ]:
prompt = "What benefits badin soft provides to it's employees?"
response = openai.Completion.create(model="text-davinci-003", prompt=prompt, temperature=0, max_tokens=1000)
response['choices'][0]['text']

### Embedding demo

### Embedding too much text

In [82]:
prompt = f"""Use the below article to answer questions about benefits in our company, if you don't know the answer write "Sorry I don't know the answer please contact HR:
     Article:
    \"\"\"
    At SuperAwsomeCompany, we prioritize the well-being and professional growth of our employees. That's why we are proud to offer a comprehensive range of benefits designed to support our team members in various aspects of their lives.

Our benefits package includes top-tier health insurance options, ensuring that you and your family have access to quality healthcare and financial security. Additionally, we provide robust retirement plans, such as our competitive 401(k) program, which allows you to save for a secure future while enjoying the added advantage of employer matching contributions.

Work-life balance is essential to us, which is why we offer generous paid time off that encompasses vacation days, sick leave, personal days, and holidays. This way, you can take the time you need to rest, rejuvenate, and spend quality time with your loved ones.

Recognizing the changing nature of work, we also embrace flexible work arrangements. Whether it's telecommuting or flexible scheduling, we understand the importance of accommodating your needs and empowering you to achieve a better work-life integration.

We are committed to the holistic well-being of our employees, which is why we provide access to employee assistance programs (EAPs). These programs offer confidential counseling services, mental health support, financial advice, and resources for childcare or eldercare, ensuring that you have the support you need during challenging times.

Investing in your professional growth is another priority for us. That's why we offer a range of opportunities for career development and training. From in-house workshops and seminars to external conferences and tuition reimbursement for further education, we're dedicated to helping you enhance your skills, succeed in your role, and progress in your career.

At SuperAwsomeCompany, we believe that our employees are our greatest asset. That's why we go above and beyond to provide a comprehensive benefits package that fosters your well-being, personal growth, and work satisfaction.
At SuperAwsomeCompany, we prioritize the well-being and professional growth of our employees. That's why we are proud to offer a comprehensive range of benefits designed to support our team members in various aspects of their lives.

Our benefits package includes top-tier health insurance options, ensuring that you and your family have access to quality healthcare and financial security. Additionally, we provide robust retirement plans, such as our competitive 401(k) program, which allows you to save for a secure future while enjoying the added advantage of employer matching contributions.

Work-life balance is essential to us, which is why we offer generous paid time off that encompasses vacation days, sick leave, personal days, and holidays. This way, you can take the time you need to rest, rejuvenate, and spend quality time with your loved ones.

Recognizing the changing nature of work, we also embrace flexible work arrangements. Whether it's telecommuting or flexible scheduling, we understand the importance of accommodating your needs and empowering you to achieve a better work-life integration.

We are committed to the holistic well-being of our employees, which is why we provide access to employee assistance programs (EAPs). These programs offer confidential counseling services, mental health support, financial advice, and resources for childcare or eldercare, ensuring that you have the support you need during challenging times.

Investing in your professional growth is another priority for us. That's why we offer a range of opportunities for career development and training. From in-house workshops and seminars to external conferences and tuition reimbursement for further education, we're dedicated to helping you enhance your skills, succeed in your role, and progress in your career.

At SuperAwsomeCompany, we believe that our employees are our greatest asset. That's why we go above and beyond to provide a comprehensive benefits package that fosters your well-being, personal growth, and work satisfaction.
At SuperAwsomeCompany, we prioritize the well-being and professional growth of our employees. That's why we are proud to offer a comprehensive range of benefits designed to support our team members in various aspects of their lives.

Our benefits package includes top-tier health insurance options, ensuring that you and your family have access to quality healthcare and financial security. Additionally, we provide robust retirement plans, such as our competitive 401(k) program, which allows you to save for a secure future while enjoying the added advantage of employer matching contributions.

Work-life balance is essential to us, which is why we offer generous paid time off that encompasses vacation days, sick leave, personal days, and holidays. This way, you can take the time you need to rest, rejuvenate, and spend quality time with your loved ones.

Recognizing the changing nature of work, we also embrace flexible work arrangements. Whether it's telecommuting or flexible scheduling, we understand the importance of accommodating your needs and empowering you to achieve a better work-life integration.

We are committed to the holistic well-being of our employees, which is why we provide access to employee assistance programs (EAPs). These programs offer confidential counseling services, mental health support, financial advice, and resources for childcare or eldercare, ensuring that you have the support you need during challenging times.

Investing in your professional growth is another priority for us. That's why we offer a range of opportunities for career development and training. From in-house workshops and seminars to external conferences and tuition reimbursement for further education, we're dedicated to helping you enhance your skills, succeed in your role, and progress in your career.

At SuperAwsomeCompany, we believe that our employees are our greatest asset. That's why we go above and beyond to provide a comprehensive benefits package that fosters your well-being, personal growth, and work satisfaction.
    \"\"\"
    Question: Give me best benefit?
"""
response = openai.Completion.create(model="text-davinci-003", prompt=prompt, temperature=0, max_tokens=1000)
response['choices'][0]['text']

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 5125 tokens (4125 in your prompt; 1000 for the completion). Please reduce your prompt; or completion length.

## Vector database

### Generating vector from text using GPT vector embeddings model

In [ ]:
text = "Simple text"
text = text.replace("\n", " ")
openai.Embedding.create(input = [text], model="text-embedding-ada-002")['data'][0]['embedding']

## Milvius connection

In [ ]:
from pymilvus import (
    connections,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

connections.connect("default", host="localhost", port="19530")

### Milvius schema creation

In [ ]:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=300),
    FieldSchema(name="file_name", dtype=DataType.VARCHAR, max_length=300),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=1536)
]
schema = CollectionSchema(fields, "articles with titles")
article_collection = Collection("articles2", schema)

### Milvius indexing

In [ ]:
index_params = {
    "metric_type":"L2",
    "index_type":"IVF_FLAT",
    "params":{"nlist":32}
}

article_collection.create_index(
    field_name="embeddings",
    index_params=index_params
)

In [ ]:
import glob

data = glob.glob("data/*.txt")
all = []
for file in data:
    with open(file, 'r') as data_file:
        lines = data_file.readlines()
        text = ' '.join([str(elem) for elem in lines])
        all.append(text)
print(' '.join([str(elem) for elem in all]))

### Embedding and inserting vectors to database

In [ ]:
import openai
openai.api_key = ""

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

i = 0
primary_keys = []
titles = []
embeddings = []
file_names = []
for file in data:
    with open(file, 'r') as data_file:
        lines = data_file.readlines()
        title = lines[0]
        text = ' '.join([str(elem) for elem in lines])
        vector_embedding = get_embedding(text)
        primary_keys.append(i)
        titles.append(title)
        file_names.append(data_file.name)
        embeddings.append(vector_embedding)
        i = i+1

article_collection.insert([primary_keys, titles, file_names, embeddings])

### Question answering using Milvius search

In [ ]:
def find_articles(question):
    query_embeddings = get_embedding(question)
    search_params = {"metric_type": "L2", "params": {"nprobe": 10}, "offset": 0}

    results = article_collection.search(
        data=[query_embeddings],
        anns_field="embeddings",
        param=search_params,
        limit=1,
        output_fields=['title', 'file_name'], # we retrieve also the URL field!
        expr=None,
        consistency_level="Strong"
    )

    for result in results[0]:
        return result.entity.get('file_name')

def get_raw_text(file):
    with open(file, 'r') as data_file:
        lines = data_file.readlines()
        return ' '.join([str(elem) for elem in lines])



def get_final_answer(question, summaries):
    prompt = f"""Use the below article to answer questions about benefits in our company, if you don't know the answer write "Sorry I don't know the answer please contact HR:
     Article:
    \"\"\"
    {summaries}
    \"\"\"
    Question: {question}?
"""
    response = openai.Completion.create(model="text-davinci-003", prompt=prompt, temperature=0, max_tokens=1000)
    return response['choices'][0]['text']

def answer(question):
    article_collection.load()
    article_candidates = find_articles(question)
    context = get_raw_text(article_candidates)
    final_answer = get_final_answer(question, context)
    print("Odogovor u fajlu " + article_candidates)
    return final_answer
answer("Give me best benefit")